# Gitlab | GitLab`Gitlab`ツールキットには、LLMエージェントがGitLabリポジトリと対話するためのツールが含まれています。このツールは[python-gitlab](https://github.com/python-gitlab/python-gitlab)ライブラリのラッパーです。> The `Gitlab` toolkit contains tools that enable an LLM agent to interact with a gitlab repository.> The tool is a wrapper for the [python-gitlab](https://github.com/python-gitlab/python-gitlab) library.## Quickstart | クイックスタート1. python-gitlabライブラリをインストールしてください   > Install the python-gitlab library2. GitLabの個人アクセストークンを作成する   > Create a Gitlab personal access token3. 環境変数を設定してください   > Set your environmental variables4. `toolkit.get_tools()`を使用してあなたのエージェントにツールを渡してください   > Pass the tools to your agent with `toolkit.get_tools()`これらのステップについては、以下で詳細に説明します。> Each of these steps will be explained in great detail below.1. **イシューを取得** - リポジトリからイシューを取得します。   > **Get Issues**- fetches issues from the repository.2. **問題を取得** - 特定の問題に関する詳細情報を取得します。   > **Get Issue**- fetches details about a specific issue.3. **Issueにコメントする** - 特定のIssueにコメントを投稿します。   > **Comment on Issue**- posts a comment on a specific issue.4. **プルリクエストを作成** - ボットの作業ブランチからベースブランチへプルリクエストを作成します。   > **Create Pull Request**- creates a pull request from the bot's working branch to the base branch.5. **ファイルを作成** - リポジトリに新しいファイルを作成します。   > **Create File**- creates a new file in the repository.6. **ファイルを読む** - リポジトリからファイルを読み込みます。   > **Read File**- reads a file from the repository.7. **ファイル更新** - リポジトリ内のファイルを更新します。   > **Update File**- updates a file in the repository.8. **ファイル削除** - リポジトリからファイルを削除します。   > **Delete File**- deletes a file from the repository.

## Setup | セットアップ

### 1. Install the `python-gitlab` library | 1. `python-gitlab` ライブラリをインストールしてください

In [ ]:
%pip install python-gitlab

### 2. Create a Gitlab personal access token | 2. GitLabの個人アクセストークンを作成する[こちらの指示に従って](https://docs.gitlab.com/ee/user/profile/personal_access_tokens.html) GitLabの個人アクセストークンを作成してください。あなたのアプリが以下のリポジトリ権限を持っていることを確認してください：> [Follow the instructions here](https://docs.gitlab.com/ee/user/profile/personal_access_tokens.html) to create a Gitlab personal access token. Make sure your app has the following repository permissions:* read\_api  > read\_api* read\_repository  > read\_repository* リポジトリに書き込む  > write\_repository### 3. Set Environmental Variables | 3. 環境変数の設定エージェントを初期化する前に、以下の環境変数を設定する必要があります：> Before initializing your agent, the following environmental variables need to be set:* **GITLAB\_URL** - GitlabがホストされているURLです。デフォルトは「https://gitlab.com」です。  > **GITLAB\_URL** - The URL hosted Gitlab. Defaults to "https://gitlab.com".* **GITLAB\_PERSONAL\_ACCESS\_TOKEN** - 前のステップで作成した個人アクセストークン  > **GITLAB\_PERSONAL\_ACCESS\_TOKEN**- The personal access token you created in the last step* **GITLAB\_REPOSITORY** - ボットが操作するGitlabのリポジトリ名です。{username}/{repo-name}の形式に従ってください。  > **GITLAB\_REPOSITORY**- The name of the Gitlab repository you want your bot to act upon. Must follow the format {username}/{repo-name}.* **GITLAB\_BRANCH** - ボットがコミットを行うブランチです。デフォルトは'main'です。  > **GITLAB\_BRANCH**- The branch where the bot will make its commits. Defaults to 'main.'* **GITLAB\_BASE\_BRANCH** - あなたのリポジトリのベースブランチで、通常は'main'または'master'です。プルリクエストはここを基点として作成されます。デフォルトは'main'です。  > **GITLAB\_BASE\_BRANCH**- The base branch of your repo, usually either 'main' or 'master.' This is where pull requests will base from. Defaults to 'main.'

## Example: Simple Agent | 例：シンプルなエージェント

In [3]:
import os

from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent_toolkits.gitlab.toolkit import GitLabToolkit
from langchain.llms import OpenAI
from langchain.utilities.gitlab import GitLabAPIWrapper

In [53]:
# Set your environment variables using os.environ
os.environ["GITLAB_URL"] = "https://gitlab.example.org"
os.environ["GITLAB_PERSONAL_ACCESS_TOKEN"] = ""
os.environ["GITLAB_REPOSITORY"] = "username/repo-name"
os.environ["GITLAB_BRANCH"] = "bot-branch-name"
os.environ["GITLAB_BASE_BRANCH"] = "main"

# This example also requires an OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

In [5]:
llm = OpenAI(temperature=0)
gitlab = GitLabAPIWrapper()
toolkit = GitLabToolkit.from_gitlab_api_wrapper(gitlab)
agent = initialize_agent(
    toolkit.get_tools(), llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [8]:
agent.run(
    "You have the software engineering capabilities of a Google Principle engineer. You are tasked with completing issues on a gitlab repository. Please look at the open issues and complete them by creating pull requests that solve the issues."
)



> Entering new AgentExecutor chain...
 I need to look at the open issues and figure out how to solve them.
Action: Get Issues
Action Input: N/A
Observation: Found 1 issues:
[{'title': 'Add tic-tac-toe game', 'number': 15}]
Thought: I need to look at the details of this issue to figure out how to solve it.
Action: Get Issue
Action Input: 15
Observation: {"title": "Add tic-tac-toe game", "body": "Create a tic-tac-toe game using HTML, CSS, and JavaScript. Create a new file called game.html and store the code there.", "comments": "[]"}
Thought: I need to create the game.html file and add the code.
Action: Create File
Action Input: game.html

test contents
Observation: Created file game.html
Thought: I need to add the code to the game.html file.
Action: Update File
Action Input: game.html

OLD <<<<
test contents
>>>> OLD
NEW <<<<
<html>
  <head>
    <title>Tic-Tac-Toe</title>
  </head>
  <body>
    <h1>Tic-Tac-Toe</h1>
    <div id="game">
      <!-- game board goes here -->
    </div>
  <

'I have created a pull request with number 12 that solves issue 15.'